## 卷积运算

卷积运算的详解可看笔记。从现代的角度来看，卷积运算可以看作被互相关运算所代替

<img src='photo/correlation.svg'>

卷积核在扫描输入，最终输出特征函数(矩阵)
$$
\begin{split}0\times0+1\times1+3\times2+4\times3=19,\\
1\times0+2\times1+4\times2+5\times3=25,\\
3\times0+4\times1+6\times2+7\times3=37,\\
4\times0+5\times1+7\times2+8\times3=43.\end{split}
$$

输出大小略小于输入大小。这是因为卷积核的宽度和高度大于1， 而卷积核只与图像中每个大小完全适合的位置进行互相关运算。 所以，输出大小等于输入大小$n_h \times n_w$减去卷积核大小$nk_h \times k_w$，即：
$$
(n_h-k_h+1) \times (n_w-k_w+1).
$$

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X,K):
    """计算二维互相关"""
    h,w = K.shape       # 卷积核大小
    Y = torch.zeros((X.shape[0]-h+1),(X.shape[1]-w+1))   # 特征函数大小
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w]*K).sum()       # 矩阵点积
    return Y
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## 卷积层

卷积层对输入和卷积核权重进行互相关运算，并在添加标量偏置后产生输出。所以，卷积层中两个被训练的参数是卷积核权重和标量偏置。

在这里随机初始化卷积核权重

$$
corr2d(input(X) \times weight(卷积核)) + b
$$

In [2]:
class Conv2D(nn.Module):
    def __init__(self,kernel_size) -> None:
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    def forward(self,x):
        return corr2d(x,self.weight)+self.bias


## 图像中的边缘检测

卷积核可以对图像进行特征提取。不同卷积核可以提取出不同的特征。接下来看一什么样卷积核可以对边缘特征进行提取

In [3]:
# 我们构造一个像素的黑白图像。中间四列为黑色（），其余像素为白色（）。
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [4]:
#接下来，我们构造一个高度为、宽度为的卷积核K。当进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零。
K = torch.tensor([[1.0, -1.0]])
K

tensor([[ 1., -1.]])

In [5]:
Y = corr2d(X, K)
Y
# 可以看到保留了边缘

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [6]:
# 我们对参数X（输入）和K（卷积核）执行互相关运算。 如下所示，输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘，其他情况的输出为。
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

现在我们将输入的二维图像转置，再进行如上的互相关运算。 其输出如下，之前检测到的垂直边缘消失了。 不出所料，这个卷积核K只可以检测垂直边缘，无法检测水平边缘。

In [7]:
# K = torch.tensor([[1.0, -1.0],[1.0,-1.0]])
# Y = corr2d(X,K)
# Y

## 学习卷积核

如果只需要寻找黑白边缘，那么以上[1,-1]的边缘检测器足以。然而，当有了更加复杂数值的卷积核或者连续的卷积层时，那么不可能手动设计滤波器(卷积核)。那么我们是否可以学习由X生成Y的卷积核呢？

In [8]:
# 构造一个二维卷积层，它具有一个输出通道和形状为(1,2)的卷积核
conv2d = nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))
lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat-Y)**2        # 误差平方和
    conv2d.zero_grad()      # 不进行求导，将模型的梯度设置为0，param.grad.zero_() 参数清零
    l.sum().backward()      # 方向传播梯度
    
    # 迭代卷积核
    conv2d.weight.data[:] -= lr*conv2d.weight.grad  
    if (i+1)%2==0:
        print(f"epoch {i+1},loss{l.sum():.3f}")

epoch 2,loss12.451
epoch 4,loss2.651
epoch 6,loss0.675
epoch 8,loss0.207
epoch 10,loss0.073


In [9]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0099, -0.9570]])

可以看出结果非常接近[1,-1]

## 特征映射与感受野

经过卷积核输出的卷积层有时称为特征映射，因为它可以被视为一个输入映射到下一层的空间维度的转换器。

在卷积神经网络中，对于某一层的任意元素x(卷积核张量上的权重)，其**感受野**是指在前向传播期间可能影响x计算的所有元素(来自所有先前层)

让我们用 图6.2.1(上图)为例来解释感受野： 给定卷积核，阴影输出元素值的感受野是输入阴影部分的四个元素。 假设之前输出为，其大小为，现在我们在其后附加一个卷积层，该卷积层以为输入，输出单个元素。 在这种情况下，输出的感受野包括的所有四个元素，而输入的感受野包括最初所有九个输入元素。 因此，当一个特征图中的任意元素需要检测更广区域的输入特征时，我们可以构建一个更深的网络。

## 填充和步幅

有时，在应用了连续的卷积之后，最终得到的输出远小于输入大小。这是由于卷积核的宽度和高度通常大于1所导致的。

比如，一个$240 \times 240$像素的图像，经过10层$5\times 5$的卷积后，将减少到$200 \times 200$像素。($(240-50+10)\times (240-50+10)$)

如此一来，原始图像的边界丢失了许多有用信息。而填充是解决此问题最有效的方法。 有时，我们可能希望大幅降低图像的宽度和高度。例如，如果我们发现原始的输入分辨率十分冗余。步幅则可以在这类情况下提供帮助。

### 1.填充

在应用多层卷积是，我们常常丢失边缘像素(边缘像素总是对其内部像素产生影响)。由于我们通常使用小卷积核，因此对于任何单个卷积，我们可能只会丢失几个像素。 但随着我们应用许多连续卷积层，累积丢失的像素数就多了。

为了解决这个问题，一种简单的方法就是填充：在输入图像的边界填充元素(通常是0)

<img src='photo/conv-pad.svg'>

那么填充之后，输出的特征矩阵有多大呢？
假设添加$p_h$行和$p_w$列
$$(n_h-k_h+p_h+1)\times (n_w-k_w+p_w+1)$$

在许多情况下，我们需要设置$p_h=k_h-1$和$p_w=k_w-1$，使输入和输出具有相同的高度和宽度。 这样可以在构建网络时更容易地预测每个图层的输出形状。

**卷积神经网络中卷积核的高度和宽度通常为奇数，例如1、3、5或7。 选择奇数的好处是，保持空间维度的同时，我们可以在顶部和底部填充相同数量的行，在左侧和右侧填充相同数量的列(k为奇数，k-1=p为偶数)。**

In [10]:
import torch
from torch import nn

# 为了方便起见，定义一个计算卷积层的函数
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d,X):
    X = X.reshape((1,1)+X.shape)
    Y = conv2d(X)       # 输出特征映射
    
    return Y.reshape(Y.shape[2:])   # 2维之后的数据是长和宽


# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列(padding=1)
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)      #Padding added to all four sides of the input. Default: 0
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d,X).shape

torch.Size([8, 8])

In [11]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))    #(5-1)/2,(3-1)/2)
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

### 2.步幅

在互相关计算的时候，卷积窗口从输入张良的左上角开始，向下、向右滑动。在前面的例子中，我们默认每次滑动一个元素。但是，有时为了高校计算或是缩减采样次数，卷积窗口可以跳过中间的位置，每次滑动多个元素。

每次滑动的数量叫做步幅(stride)

<img src = 'photo/conv-stride.svg'>

如图为垂直步幅为3，水平步幅为2 的二位互相关运算

通常垂直步幅$s_h$,和水平步幅$s_w$时，输出形状为：
$$\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor.$$

如果我们设置了$p_h=k_h-1$和$p_w=k_w-1$，则输出形状将简化为$\lfloor(n_h+s_h-1)/s_h\rfloor \times \lfloor(n_w+s_w-1)/s_w\rfloor$。 更进一步，如果输入的高度和宽度可以被垂直和水平步幅整除，则输出形状将为$(n_h/s_h) \times (n_w/s_w)$。

In [12]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [13]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

## 多输入、输出通道

对于一张彩色图像，是具有标准的RGB通道来代表红、绿、蓝。在之前只展示了单个输入输出通道，这使得可以将输入输出、卷积核看作是二位张量

当增加通道的时候，我们的输入和隐藏的表示都变成了三维张量。例如，每个RGB输入图像具有3\*h\*w的形状


### 1.多输入通道

当输入包含多个通道的时候，**需要构造一个与输入数据具有相同输入通道数的卷积核**，以便与输入数据进行互相关运算。

在这之后，每一个卷积核与其对应通道输入进行互相关运算，在对通道求和得到二维张量。这就是多通道输入和多输入通道卷积核之间进行二维互相关运算的结果

<img src = 'photo/conv-multi-in.svg'>

In [14]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X,K):
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K)) # x,k就是单通道二维张量

In [15]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
X

tensor([[[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]],

        [[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]])

In [16]:
K

tensor([[[0., 1.],
         [2., 3.]],

        [[1., 2.],
         [3., 4.]]])

In [17]:

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

### 2.多通道输出

到目前为止，不论有多少输入通道，目前讨论的情况只有一个输出通道。

在最流行的神经网络架构中，随着神经网络层数的加深，我们常会增加输出通道的维数，通过减少空间分辨率以获得更大的通道深度。**直观的说，我们可以将每个通道看作是对不同特征的相应**

而现实可能更为复杂一些，因为每个通道不是独立学习的，而是为了共同使用而优化的。因此，多输出通道并不仅是学习多个单通道的检测器。

用$c_i和c_0$分别表示输入和输出通道的数目，并让$k_h和k_w$为卷积核的高度和宽度。为了获得多个通道的输出，我们可以为每个输出通道创建一个形状为$c_i\times k_h \times k_w$的卷积核张量，这样卷积核的形状为$c_0 \times c_i\times k_h \times k_w$。在互相关运算中，每个输出通道先获取所有输入通道，再以对应该输出通道的卷积核计算出结果

<img src = 'photo/多通道.png'>

In [18]:
def corr2d_multi_in_out(X,K):
    # 迭代 K 的第0个维度，每次都对输入X执行互相关运算
    # 最后将所有结果都叠加在一起
    
    return torch.stack([corr2d_multi_in(X,k) for k in K],0) 

In [19]:
K,K+1,K+2

(tensor([[[0., 1.],
          [2., 3.]],
 
         [[1., 2.],
          [3., 4.]]]),
 tensor([[[1., 2.],
          [3., 4.]],
 
         [[2., 3.],
          [4., 5.]]]),
 tensor([[[2., 3.],
          [4., 5.]],
 
         [[3., 4.],
          [5., 6.]]]))

In [20]:
K = torch.stack((K, K + 1, K + 2), 0)   # 三个输出通道
K.shape,K

(torch.Size([3, 2, 2, 2]),
 tensor([[[[0., 1.],
           [2., 3.]],
 
          [[1., 2.],
           [3., 4.]]],
 
 
         [[[1., 2.],
           [3., 4.]],
 
          [[2., 3.],
           [4., 5.]]],
 
 
         [[[2., 3.],
           [4., 5.]],
 
          [[3., 4.],
           [5., 6.]]]]))

In [23]:
# torch.stack()
# 官方解释：沿着一个新维度对输入张量序列进行连接。 序列中所有的张量都应该为相同形状。

# 浅显说法：把多个2维的张量凑成一个3维的张量；多个3维的凑成一个4维的张量…以此类推，也就是在增加新的维度进行堆叠。

T1 = torch.tensor([[1, 2, 3],
        		[4, 5, 6],
        		[7, 8, 9]])
# 假设是时间步T2的输出
T2 = torch.tensor([[10, 20, 30],
        		[40, 50, 60],
        		[70, 80, 90]])
print(torch.stack((T1,T2),dim=0),torch.stack((T1,T2),dim=0).shape)
print(torch.stack((T1,T2),dim=1),torch.stack((T1,T2),dim=1).shape)
print(torch.stack((T1,T2),dim=2).shape)
# print(torch.stack((T1,T2),dim=3).shape)
# outputs:
# torch.Size([2, 3, 3])
# torch.Size([3, 2, 3])
# torch.Size([3, 3, 2])
# '选择的dim>len(outputs)，所以报错'
# IndexError: Dimension out of range (expected to be in range of [-3, 2], but got 3)



tensor([[[ 1,  2,  3],
         [ 4,  5,  6],
         [ 7,  8,  9]],

        [[10, 20, 30],
         [40, 50, 60],
         [70, 80, 90]]]) torch.Size([2, 3, 3])
tensor([[[ 1,  2,  3],
         [10, 20, 30]],

        [[ 4,  5,  6],
         [40, 50, 60]],

        [[ 7,  8,  9],
         [70, 80, 90]]]) torch.Size([3, 2, 3])
torch.Size([3, 3, 2])


In [24]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

### 1*1卷积核

卷积的本质是有效提取相邻像素之间的相关特征，而1*1的卷积核显然没有这个作用。看起来似乎没有什么意义。但是在复杂深层网络的设计中，该卷积核十分流行

如下图：我们可以将1*1卷积层看作是在每个像素位置应用的全连接层，以$c_i$个输入值转换为$c_0$个输出值。因为这仍然是一个卷积层，所以跨像素的权重是一致的。 同时，1\*1卷积层需要的权重维度为$c_0 \times c_i$，再额外加上一个偏置。

<img src='photo/conv-1x1.svg'>

In [25]:
def corr2d_multi_in_out_1x1(X,K):
    c_i,h,w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i,h*w))    # 每一个通道变成一个向量
    print(X)
    K = K.reshape((c_o,c_i))    # 变成二维向量
    print(K)
    
    # 全连接层的矩阵乘法
    Y = torch.matmul(K,X)       # 第一行×第一列 三个卷积核对对每个通道的第一个像素进行扫描
    print(Y)                    # 第一行就是第一输出通道的值，第二行就是第二个
    return Y.reshape((c_o,h,w))

In [26]:
X = torch.normal(0,1, (3,3,3))
K = torch.normal(0,1,(2,3,1,1))
X,K

(tensor([[[ 0.4201,  0.5162, -1.2962],
          [ 1.2129, -0.3062,  0.0093],
          [-0.2185, -1.0849,  0.2297]],
 
         [[-1.2186, -0.6411, -0.0949],
          [-0.1090, -0.6125,  1.3919],
          [ 1.2818, -0.7393,  0.1051]],
 
         [[ 1.4196,  1.3359,  0.1425],
          [-0.5855,  0.1980, -1.5347],
          [ 0.3052, -0.8499,  0.9355]]]),
 tensor([[[[-1.1509]],
 
          [[-0.8590]],
 
          [[-0.3075]]],
 
 
         [[[-0.9089]],
 
          [[ 0.9439]],
 
          [[ 1.4466]]]]))

In [27]:
corr2d_multi_in_out_1x1(X, K)

tensor([[ 0.4201,  0.5162, -1.2962,  1.2129, -0.3062,  0.0093, -0.2185, -1.0849,
          0.2297],
        [-1.2186, -0.6411, -0.0949, -0.1090, -0.6125,  1.3919,  1.2818, -0.7393,
          0.1051],
        [ 1.4196,  1.3359,  0.1425, -0.5855,  0.1980, -1.5347,  0.3052, -0.8499,
          0.9355]])
tensor([[-1.1509, -0.8590, -0.3075],
        [-0.9089,  0.9439,  1.4466]])
tensor([[ 0.1268, -0.4542,  1.5294, -1.1222,  0.8177, -0.7343, -0.9435,  2.1451,
         -0.6424],
        [ 0.5215,  0.8582,  1.2948, -2.0523, -0.0134, -0.9147,  1.8499, -0.9411,
          1.2437]])


tensor([[[ 0.1268, -0.4542,  1.5294],
         [-1.1222,  0.8177, -0.7343],
         [-0.9435,  2.1451, -0.6424]],

        [[ 0.5215,  0.8582,  1.2948],
         [-2.0523, -0.0134, -0.9147],
         [ 1.8499, -0.9411,  1.2437]]])

In [28]:
corr2d_multi_in_out(X, K)

tensor([[[ 0.1268, -0.4542,  1.5294],
         [-1.1222,  0.8177, -0.7343],
         [-0.9435,  2.1451, -0.6424]],

        [[ 0.5215,  0.8582,  1.2948],
         [-2.0523, -0.0134, -0.9147],
         [ 1.8499, -0.9411,  1.2437]]])

## 池化层（汇聚层）

我们的机器学习任务通常会跟全局图像的问题有关（图像是否包含一只猫），所以我们最后一层的神经元应该包含整个输入的全局敏感。在我们进行处理图像的时候，我们希望逐渐降低隐藏表示的空间分辨率、聚集信息（卷积核扫出的局部特征），这样随着我们在神经网络中层叠的上升，每个神经元对其敏感的感受野(上一层影响下一层的元素)就越大(**这句话什么意思呢？就像最大池化层，只保留了重要的信息，上一层的元素是其感受野，且对其非常敏感**)。通过逐渐聚合信息，生成越来越粗糙的映射，最终实现学习全局表示的目标，同时将卷积层的所有有时保留在中间层


此外，当检测较底层的特征时（例如 6.2节中所讨论的边缘），我们通常希望这些特征保持某种程度上的平移不变性。例如，如果我们拍摄黑白之间轮廓清晰的图像X，并将整个图像向右移动一个像素，即Z[i, j] = X[i, j + 1]，则新图像Z的输出可能大不相同。而在现实中，随着拍摄角度的移动，任何物体几乎不可能发生在同一像素上。即使用三脚架拍摄一个静止的物体，由于快门的移动而引起的相机振动，可能会使所有物体左右移动一个像素（除了高端相机配备了特殊功能来解决这个问题）。

本节将介绍汇聚（pooling）层，它具有双重目的：降低卷积层对位置的敏感性，同时降低对空间降采样表示的敏感性。

降低过拟合的概率

### 最大池化层和平均池化层

计算过程与卷积层类似。不过池化层不包含参数，相反，池运算时确定的，我们通常计算汇聚窗口中所有元素的最大值或平均值。这些操作分别称为最大池化层和平均池化层

<img src = 'photo/pooling.svg'>

In [29]:
import torch
from torch import nn
from d2l import torch as d2l

def pool2d(X,pool_size,mode='max'):
    p_h,p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1)) #与卷积层类似
    
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode=='max':
                Y[i,j] = X[i:i+p_h,j:j+p_w].max()
            else:
                Y[i,j] = X[i:i+p_h,j:j+p_w].mean()
    return Y


In [30]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X,(2,2))

tensor([[4., 5.],
        [7., 8.]])

In [31]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

### 填充与步幅、多通道

In [32]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [33]:
"""
默认情况下，深度学习框架中的步幅与汇聚窗口的大小相同。 
因此，如果我们使用形状为(3, 3)的汇聚窗口，那么默认情况下，我们得到的步幅形状为(3, 3)。
"""
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

In [34]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [35]:
pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

在处理多通道输入数据时，汇聚层在每个输入通道上单独运算，而不是像卷积层一样在通道上对输入进行汇总。 这意味着汇聚层的输出通道数与输入通道数相同。 下面，我们将在通道维度上连结张量X和X + 1，以构建具有2个通道的输入。

In [36]:
X = torch.cat((X, X + 1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [37]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])